In [4]:
from qiskit.providers.fake_provider import FakeLima, FakeBelem
from qiskit.providers.aer.noise import NoiseModel
from qiskit.quantum_info import average_gate_fidelity, process_fidelity, PTM

from qiskit_aer import AerSimulator, QasmSimulator
import sys
sys.path.append('../tutorials/')
from noise_utils import AddNoise

from blackwater.data.utils import (
    get_backend_properties_v1,
)

import qiskit, random, os
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit_aer.noise import depolarizing_error, coherent_unitary_error, mixed_unitary_error, thermal_relaxation_error
from qiskit.providers.aer.noise import NoiseModel, mixed_unitary_error, pauli_error
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import CXGate, RXGate, IGate, ZGate, RZXGate, RZZGate
from qiskit.providers.fake_provider import FakeMontreal, FakeLima
import itertools

from qiskit_aer import AerSimulator, QasmSimulator

import numpy as np

In [13]:
FakeLima().properties().to_dict()

{'backend_name': 'ibmq_lima',
 'backend_version': '1.0.5',
 'last_update_date': datetime.datetime(2021, 3, 15, 0, 36, 3, tzinfo=tzoffset(None, -14400)),
 'qubits': [[{'date': datetime.datetime(2021, 3, 15, 0, 9, 11, tzinfo=tzoffset(None, -14400)),
    'name': 'T1',
    'unit': 'us',
    'value': 59.69864328663569},
   {'date': datetime.datetime(2021, 3, 15, 0, 9, 44, tzinfo=tzoffset(None, -14400)),
    'name': 'T2',
    'unit': 'us',
    'value': 93.55584184359311},
   {'date': datetime.datetime(2021, 3, 15, 0, 36, 3, tzinfo=tzoffset(None, -14400)),
    'name': 'frequency',
    'unit': 'GHz',
    'value': 5.02968554992376},
   {'date': datetime.datetime(2021, 3, 15, 0, 36, 3, tzinfo=tzoffset(None, -14400)),
    'name': 'anharmonicity',
    'unit': 'GHz',
    'value': -0.33574155258829724},
   {'date': datetime.datetime(2021, 3, 15, 0, 8, 49, tzinfo=tzoffset(None, -14400)),
    'name': 'readout_error',
    'unit': '',
    'value': 0.026100000000000012},
   {'date': datetime.datetime(202

In [14]:
dir(FakeLima().configuration())

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_channel_qubit_map',
 '_control_channels',
 '_data',
 '_get_channel_prefix_index',
 '_parse_channels',
 '_qubit_channel_map',
 'acquire',
 'acquisition_latency',
 'backend_name',
 'backend_version',
 'basis_gates',
 'channels',
 'conditional',
 'conditional_latency',
 'control',
 'control_channels',
 'coupling_map',
 'credits_required',
 'default_rep_delay',
 'describe',
 'description',
 'discriminators',
 'drive',
 'dt',
 'dtm',
 'dynamic_reprate_enabled',
 'from_dict',
 'gates',
 'get_channel_qubits',
 'get_qubit_channels',
 'hamiltonian',
 'local',
 'max_experiments',
 'max_shots',
 'm

In [9]:
noise_model = NoiseModel.from_backend(FakeLima())
dir(noise_model)

/var/folders/94/svsqs2bx4j5ct0r0f6jyr1qm0000gn/T/ipykernel_39087/2581628487.py:1: DeprecationWarning: All fake backend instances based on real device snapshots (`FakeVigo`,`FakeSherbrooke`,...) have been migrated to the `qiskit_ibm_runtime` package. These classes are deprecated as of qiskit 0.46.0 and will be removed in qiskit 1.0.0. To migrate your code, run `pip install qiskit-ibm-runtime` and use `from qiskit_ibm_runtime.fake_provider import FakeExample` instead of `from qiskit.providers.fake_provider import FakeExample`. If you are using a custom fake backend implementation, you don't need to take any action.
  noise_model = NoiseModel.from_backend(FakeLima())


['_1qubit_instructions',
 '_2qubit_instructions',
 '_3qubit_instructions',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_all_qubit_quantum_errors_equal',
 '_basis_gates',
 '_check_number_of_qubits',
 '_custom_noise_passes',
 '_default_quantum_errors',
 '_default_readout_error',
 '_instruction_names_labels',
 '_keys2str',
 '_local_quantum_errors',
 '_local_quantum_errors_equal',
 '_local_readout_errors',
 '_noise_instructions',
 '_noise_qubits',
 '_pass_manager',
 '_readout_errors_equal',
 'add_all_qubit_quantum_error',
 'add_all_qubit_readout_error',
 'add_basis_gates',
 'add_quantum_error',
 'add_readout_error',
 'basis_gates',
 'from_backend',
 'from_backend_prop

In [4]:
noise_model = NoiseModel.from_backend(FakeLima())
noise_dict = noise_model.to_dict()['errors']
noise_dict

/var/folders/94/svsqs2bx4j5ct0r0f6jyr1qm0000gn/T/ipykernel_39087/3895923856.py:1: DeprecationWarning: All fake backend instances based on real device snapshots (`FakeVigo`,`FakeSherbrooke`,...) have been migrated to the `qiskit_ibm_runtime` package. These classes are deprecated as of qiskit 0.46.0 and will be removed in qiskit 1.0.0. To migrate your code, run `pip install qiskit-ibm-runtime` and use `from qiskit_ibm_runtime.fake_provider import FakeExample` instead of `from qiskit.providers.fake_provider import FakeExample`. If you are using a custom fake backend implementation, you don't need to take any action.
  noise_model = NoiseModel.from_backend(FakeLima())


[{'type': 'qerror',
  'id': 'dff3465c304b4ba381ce156a047c8ee4',
  'operations': ['id'],
  'instructions': [[{'name': 'kraus',
     'params': [array([[-0.99997945+0.j,  0.        +0.j],
             [ 0.        +0.j, -0.99968168+0.j]]),
      array([[-0.00641101+0.j,  0.        +0.j],
             [ 0.        +0.j,  0.00641292+0.j]]),
      array([[0.        +0.j, 0.02440096+0.j],
             [0.        +0.j, 0.        +0.j]])],
     'qubits': [0]}]],
  'probabilities': [1.0],
  'gate_qubits': [(0,)]},
 {'type': 'qerror',
  'id': '5c860cbb0be144e19a1485972fa21c10',
  'operations': ['id'],
  'instructions': [[{'name': 'id', 'qubits': [0]},
    {'name': 'kraus',
     'params': [array([[-0.99997658+0.j,  0.        +0.j],
             [ 0.        +0.j, -0.99976255+0.j]]),
      array([[-0.00684392+0.j,  0.        +0.j],
             [ 0.        +0.j,  0.00684538+0.j]]),
      array([[0.        +0.j, 0.02068766+0.j],
             [0.        +0.j, 0.        +0.j]])],
     'qubits': [0]}],
  

In [30]:
noise_model = NoiseModel.from_backend(FakeLima())
noise_model._local_readout_errors

{(0,): ReadoutError([[0.9882 0.0118]
  [0.0404 0.9596]]),
 (1,): ReadoutError([[0.9888 0.0112]
  [0.0288 0.9712]]),
 (2,): ReadoutError([[0.9928 0.0072]
  [0.026  0.974 ]]),
 (3,): ReadoutError([[0.9752 0.0248]
  [0.0782 0.9218]]),
 (4,): ReadoutError([[0.9808 0.0192]
  [0.0958 0.9042]])}

In [20]:
noise_model = NoiseModel.from_backend(FakeLima())

gate_set = ['cx', 'x', 'sx']
for g in gate_set:
    all_infids = [1-average_gate_fidelity(x) for x in noise_model._local_quantum_errors[g].values()]
    print(np.mean(all_infids), np.std(all_infids)/len(all_infids))

0.012045820265615542 0.0008869005556284779
0.0004377446555523301 6.374067470287409e-05
0.0004377446555523301 6.374067470287409e-05


In [21]:
noise_model = NoiseModel.from_backend(FakeBelem())

gate_set = ['cx', 'x', 'sx']
for g in gate_set:
    all_infids = [1-average_gate_fidelity(x) for x in noise_model._local_quantum_errors[g].values()]
    print(np.mean(all_infids), np.std(all_infids)/len(all_infids))

0.013986040396766028 0.00026547523201857334
0.0004076419666134257 2.9369164240261466e-05
0.0004076419666134257 2.9369164240261466e-05


In [23]:
# Local, coherent noise
backend_ideal = QasmSimulator() # Noiseless
backend_noisy_coherent, noise_model = AddNoise(backend=FakeLima()).add_coherent_noise(seed=0, theta=np.pi * 0.04, uniform=False, add_depolarization=True)

gate_set = ['cx', 'x', 'sx']
for g in gate_set:
    all_infids = [1-average_gate_fidelity(x) for x in noise_model._local_quantum_errors[g].values()]
    print(np.mean(all_infids), np.std(all_infids)/len(all_infids))

random seed fixed to 0
thetas [0.06896594 0.08987335 0.07574548 0.06847204 0.05323803 0.08116545
 0.05498883 0.1120635 ]
0.019089432060830455 0.0015271227059501641
0.0004377446555523301 6.374067470287409e-05
0.0004377446555523301 6.374067470287409e-05


/Users/haoranliao/anaconda3/envs/mlqem/lib/python3.8/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/haoranliao/anaconda3/envs/mlqem/lib/python3.8/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [24]:
# Local, coherent noise
backend_ideal = QasmSimulator() # Noiseless
backend_noisy_coherent, noise_model = AddNoise(backend=FakeBelem()).add_coherent_noise(seed=0, theta=np.pi * 0.04, uniform=False, add_depolarization=True)

gate_set = ['cx', 'x', 'sx']
for g in gate_set:
    all_infids = [1-average_gate_fidelity(x) for x in noise_model._local_quantum_errors[g].values()]
    print(np.mean(all_infids), np.std(all_infids)/len(all_infids))

random seed fixed to 0
thetas [0.06896594 0.08987335 0.07574548 0.06847204 0.05323803 0.08116545
 0.05498883 0.1120635 ]
0.02046203937948117 0.00031919860632930944
0.0004076419666134257 2.9369164240261466e-05
0.0004076419666134257 2.9369164240261466e-05


/Users/haoranliao/anaconda3/envs/mlqem/lib/python3.8/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/haoranliao/anaconda3/envs/mlqem/lib/python3.8/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [26]:
tmp = get_backend_properties_v1(FakeBelem())
tmp

{'name': 'fake_belem',
 'gates_set': ['rz', 'x', 'id', 'cx', 'sx', 'reset'],
 'num_qubits': 5,
 'qubits_props': {0: {'index': 0,
   't1': 8.857848970762537e-05,
   't2': 0.00010679794866226273,
   'readout_error': 0.04139999999999999},
  1: {'index': 1,
   't1': 7.805043996837796e-05,
   't2': 6.376578004446572e-05,
   'readout_error': 0.03200000000000003},
  2: {'index': 2,
   't1': 6.932578590725382e-05,
   't2': 3.8623766504632775e-05,
   'readout_error': 0.02400000000000002},
  3: {'index': 3,
   't1': 5.361400062698815e-05,
   't2': 5.6469814445204655e-05,
   'readout_error': 0.03400000000000003},
  4: {'index': 4,
   't1': 0.00011055756502325884,
   't2': 0.0001316820368187947,
   'readout_error': 0.02059999999999995}},
 'gate_props': {'id_0': {'index': 'id_0',
   'gate_error': 0.00023078387665829674,
   'gate_length': 35.55555555555556},
  'id_1': {'index': 'id_1',
   'gate_error': 0.0004110884185250172,
   'gate_length': 35.55555555555556},
  'id_2': {'index': 'id_2',
   'gate_

In [27]:
t1 = []
t2 = []
readout = []
for val in tmp['qubits_props'].values():
    t1.append(val['t1'])
    t2.append(val['t2'])
    readout.append(val['readout_error'])
print(np.mean(t1))
print(np.mean(t2))
print(np.mean(readout))

8.002525624670083e-05
7.946786929507211e-05
0.030400000000000003


In [70]:
sx = []
x = []
cx = []
reset = []
for key, val in tmp['gate_props'].items():
    if 'sx' == key[:2]:
        sx.append(val['gate_error'])
    elif 'x' == key[0]:
        x.append(val['gate_error'])
    elif 'cx' == key[:2]:
        cx.append(val['gate_error'])
    elif 'reset' in key:
        reset.append(val['gate_error'])
print(np.mean(sx))
print(np.mean(x))
print(np.mean(cx))
print(np.mean(reset))

0.0003886014396357395
0.0003886014396357395
0.013986040396766104
0.0


In [77]:
backend = FakeBelem()
properties = get_backend_properties_v1(backend)

# Local, coherent noise
backend_ideal = QasmSimulator() # Noiseless
backend_noisy_coherent, noise_model = AddNoise(backend=backend).add_coherent_noise(seed=0, theta=np.pi * 0.04, uniform=False, add_depolarization=True)

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy_coherent = {'shots': 10000, 'backend': backend_noisy_coherent, 'name': 'noisy_coherent'}

random seed fixed to 0
thetas [0.06896594 0.08987335 0.07574548 0.06847204 0.05323803 0.08116545
 0.05498883 0.1120635 ]


/Users/haoranliao/anaconda3/envs/mlqem/lib/python3.8/site-packages/numpy/linalg/linalg.py:2139: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/haoranliao/anaconda3/envs/mlqem/lib/python3.8/site-packages/numpy/linalg/linalg.py:2139: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [76]:
np.mean([0.06896594, 0.08987335, 0.07574548, 0.06847204, 0.05323803, 0.08116545,
 0.05498883, 0.1120635])

0.0755640775

In [78]:
np.mean([0.06896594, 0.08987335, 0.07574548, 0.06847204, 0.05323803, 0.08116545, 0.05498883, 0.1120635])

0.0755640775